In [2]:
# COVERAGE OF DATASET OBSTACLE TRAJECTORIES

from debugging_utils import obstacle_distribution_metric
import os
import h5py

demo_names = ["10obs_TS2+_lammr0_model_3090_drop_FIXED_1020ro_03roCl_4d_100u_cG_0velTime_12Vel"]

# params:
subfolder = "single_render"
# end params
for demo_name in demo_names:
        
    demo_path = os.path.join("/scratch/sghani2/hallucination_new/LfH_demo/",subfolder, demo_name)
    h5_file = h5py.File(os.path.join(demo_path, "LfH_demo.h5"), "r")

    obs_trajs = h5_file["sd_obs_loc"][:]
    robot_psi = h5_file["sd_psi"][:]
    robot_traj = h5_file["sd_traj"][:]

    h5_file.close()
    print("demo_name:", demo_name)
    fig = obstacle_distribution_metric(obs_trajs, robot_psi, robot_traj, visualize=False, use_raw_values=True, SPEED_BOUNDS=[1, 2], RADIUS_BOUNDS=[0.65, 2.5], THETA_RESOLUTION=5, DIRECTION_RESOLUTION=5)

demo_name: 10obs_TS2+_lammr0_model_3090_drop_FIXED_1020ro_03roCl_4d_100u_cG_0velTime_12Vel
(Joint) Coverage over 4 metric(s):
['pos_angle' 'pos_dist' 'vel_angle' 'vel_speed'] : 70.23445784149382
---------
(Joint) Coverage over 3 metric(s):
['pos_dist' 'vel_angle' 'vel_speed'] : 100.0
['pos_angle' 'vel_angle' 'vel_speed'] : 99.50016206349477
['pos_angle' 'pos_dist' 'vel_speed'] : 100.0
['pos_angle' 'pos_dist' 'vel_angle'] : 95.45387206052285
---------
(Joint) Coverage over 2 metric(s):
['vel_angle' 'vel_speed'] : 100.0
['pos_dist' 'vel_speed'] : 100.0
['pos_dist' 'vel_angle'] : 100.0
['pos_angle' 'vel_speed'] : 100.0
['pos_angle' 'vel_angle'] : 100.0
['pos_angle' 'pos_dist'] : 100.0
---------
(Joint) Coverage over 1 metric(s):
['vel_speed'] : 100.0
['vel_angle'] : 100.0
['pos_dist'] : 100.0
['pos_angle'] : 100.0
---------


In [16]:
from debugging_utils import obstacle_distribution_metric, quat_to_psi, rotate_origin_only
import pickle
import numpy as np

def setup_timesteps_traj(knot_start=-3, knot_end=18):
    ref_pt_ts = np.arange(knot_start+2,  knot_end -2)* 0.15
    reference_pt_idx = np.round(ref_pt_ts * 50).astype(np.int32)

    ref_pt_idx0 = reference_pt_idx - reference_pt_idx[0]
    full_traj_ts = []
    for i in range(len(ref_pt_idx0) -2):
        x = np.linspace(ref_pt_ts[i], ref_pt_ts[i+1],ref_pt_idx0[i+1]-ref_pt_idx0[i] +1)
        full_traj_ts.extend(x[:-1])
    x = np.linspace(ref_pt_ts[len(ref_pt_idx0) -2], 
                    ref_pt_ts[len(ref_pt_idx0) -1], 
                    ref_pt_idx0[len(ref_pt_idx0) -1] -ref_pt_idx0[len(ref_pt_idx0) -2] +1)
    full_traj_ts.extend(x)
    full_traj_ts = np.array(full_traj_ts)
    traj_start = reference_pt_idx[1] - reference_pt_idx[0] # 8
    traj_end = reference_pt_idx[-2] - reference_pt_idx[0] + 1   # +1: for exclusion # 114
    timesteps_traj = full_traj_ts[traj_start: traj_end]
    full_traj_ref_pt_ts = full_traj_ts[reference_pt_idx - reference_pt_idx[0]]

    return reference_pt_idx, timesteps_traj, full_traj_ts, full_traj_ref_pt_ts


dyna_lflh_eval = "/projects/RobotiXX/saad/hallucination/LfH_eval/dyna_lflh/2xlen_3obs_lflh_model_3000/LfH_eval.p"
with open(dyna_lflh_eval, "rb") as f:
    dyna_lflh_eval = pickle.load(f)

for k in dyna_lflh_eval.keys():
    print(k, dyna_lflh_eval[k].shape)

obs_loc (4734, 3, 2)
obs_size (4734, 3, 2)
obs_vel (4734, 3, 2)
obs_acc (4734,)
traj (4734, 2, 233, 2)
goal (4734, 233, 2)
raw_ori (4734, 233, 4)
raw_traj (4734, 2, 233, 3)
recon_traj (4734, 2, 233, 2)
full_traj (4734, 4, 249)
obs_time (4734,)
debug_obs_loc_evolved (4734, 3, 34, 2)
debug_obs_size_evolved (4734, 3, 34, 2)
debug_obs_loc_mu (4734, 3, 2)
debug_obs_loc_logvar (4734, 3, 2)
debug_obs_size_mu (4734, 3, 2)
debug_obs_size_logvar (4734, 3, 2)
debug_obs_vel_mu (4734, 3, 2)
debug_obs_vel_logvar (4734, 3, 2)
cmd (4734, 2, 233)


In [18]:
reference_pt_idx, timesteps_traj, full_traj_ts, full_traj_ref_pt_ts = setup_timesteps_traj(knot_end=35)

obs_locs = dyna_lflh_eval["obs_loc"]
obs_vels = dyna_lflh_eval["obs_vel"]
obs_trajs = obs_locs[:, :, None] + obs_vels[:, :, None] * timesteps_traj[None, None, :, None]
obs_trajs = obs_trajs[:, :, ::5]
robot_quats = dyna_lflh_eval["raw_ori"][:, ::5]
robot_trajs = dyna_lflh_eval["traj"][:,:, ::5]

robot_psis = quat_to_psi(robot_quats)
robot_psis = robot_psis.reshape(-1, 1, 47)

obs_trajs = obs_trajs.reshape(-1, 1, 1, 3, 47, 2)
robot_trajs = robot_trajs.reshape(-1,1, 2, 47, 2)


print(robot_trajs.shape, robot_psis.shape, obs_trajs.shape)
obstacle_distribution_metric(obs_trajs, robot_psis, robot_trajs, visualize=False, use_raw_values=True, SPEED_BOUNDS=[0, 1], RADIUS_BOUNDS=[0.65, 2.5], THETA_RESOLUTION=5, DIRECTION_RESOLUTION=5)


(4734, 1, 2, 47, 2) (4734, 1, 47) (4734, 1, 1, 3, 47, 2)
(Joint) Coverage over 4 metric(s):
['pos_angle' 'pos_dist' 'vel_angle' 'vel_speed'] : 11.051114197749786
---------
(Joint) Coverage over 3 metric(s):
['pos_dist' 'vel_angle' 'vel_speed'] : 69.64016517008587
['pos_angle' 'vel_angle' 'vel_speed'] : 49.30312697248333
['pos_angle' 'pos_dist' 'vel_speed'] : 74.60837648292588
['pos_angle' 'pos_dist' 'vel_angle'] : 40.494414869976595
---------
(Joint) Coverage over 2 metric(s):
['vel_angle' 'vel_speed'] : 91.40722291407222
['pos_dist' 'vel_speed'] : 89.47368421052632
['pos_dist' 'vel_angle'] : 86.51766402307138
['pos_angle' 'vel_speed'] : 99.12826899128268
['pos_angle' 'vel_angle'] : 89.05986113717395
['pos_angle' 'pos_dist'] : 89.40158615717375
---------
(Joint) Coverage over 1 metric(s):
['vel_speed'] : 100.0
['vel_angle'] : 100.0
['pos_dist'] : 89.47368421052632
['pos_angle'] : 100.0
---------


(array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0.

In [ ]:
obstacle_distribution_metric(obs_trajs, robot_psi, robot_traj, visualize=False, use_raw_values=True, SPEED_BOUNDS=[1, 2], RADIUS_BOUNDS=[0.65, 2.5], THETA_RESOLUTION=5, DIRECTION_RESOLUTION=5)